In [64]:
import pandas as pd
import numpy as np
import pyodbc
import subprocess
import io
import os

import matplotlib as plt

# Ignore all warnings
import warnings
warnings.filterwarnings('ignore')


In [ ]:
# Path to your Access database
db_path = r'C:\Users\mehak.rafiq.ASKARIBANK\Documents\Projects\model_data\Daily_Dashboard_NTB\Data\Account_Holders_Weekly_be.accdb'

# Connection string
conn_str = (
    r"DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};"
    f"DBQ={db_path};"
)

try:
    # Establish a connection
    with pyodbc.connect(conn_str) as conn:
        cursor = conn.cursor()

        # Retrieve table names
        cursor.tables()
        table_names = [row.table_name for row in cursor if row.table_type == 'TABLE']
        print("Tables in the database:", table_names)

        # Load each table into a Pandas DataFrame
        tables_data = {}
        for table_name in table_names:
            query = f"SELECT * FROM [{table_name}]"
            tables_data[table_name] = pd.read_sql(query, conn)
            print(f"Loaded table: {table_name}")

except pyodbc.Error as e:
    print("Database connection error:", e)
except Exception as e:
    print("Error:", e)

In [ ]:
# Create a new folder called './Data/CSVs' if it doesn't exist
# Creat in a one directory up
os.makedirs('./Data/CSVs', exist_ok=True)


# Save each table as a separate CSV file in the './Data/CSVs' folder
for table_name, df in tables_data.items():
    csv_path = os.path.join('./Data/CSVs', f"{table_name}.csv")
    df.to_csv(csv_path, index=False)
    print(f"Saved {table_name} to {csv_path}")

In [ ]:
# Path to ntb_reg_tbl
ntb_reg_tbl_csv_path = os.path.join('./Data/CSVs', 'NTB_Reg_Summary.csv')

# Load the CSV file into a DataFrame
ntb_reg_tbl_df = pd.read_csv(ntb_reg_tbl_csv_path)

# Print Shape of the DataFrame
print(f"Shape of the DataFrame: {ntb_reg_tbl_df.shape}")

# Display the DataFrame
ntb_reg_tbl_df.head()

In [ ]:
# Find Unique values in Login_Bracket
print(ntb_reg_tbl_df['Login_Bracket'].unique())



In [ ]:
# Convert date columns to datetime
ntb_reg_tbl_df['Registration_Date'] = pd.to_datetime(ntb_reg_tbl_df['Registration_Date'])
ntb_reg_tbl_df['Open_Date_'] = pd.to_datetime(ntb_reg_tbl_df['Open_Date_'])

# Create mask for records where Registration_Date is older than Open_Date_
older_reg_mask = ntb_reg_tbl_df['Registration_Date'] < ntb_reg_tbl_df['Open_Date_']

# Update Registration_Remarks where mask is True
ntb_reg_tbl_df.loc[older_reg_mask, 'Registration_Remarks'] = 'Already Registered'

# Count number of updates
num_updates = older_reg_mask.sum()

print(f"Number of records updated to 'Already Registered': {num_updates}")


In [ ]:
# Len of CUST_AC_NO
print(len(ntb_reg_tbl_df['CUST_AC_NO'].unique()))

# Percentage of records updated to 'Already Registered'
print(f"Percentage of records updated to 'Already Registered': {num_updates / len(ntb_reg_tbl_df) * 100}%")


In [ ]:
# Convert Last_Login_Da to datetime
ntb_reg_tbl_df['Last_Login_Date'] = pd.to_datetime(ntb_reg_tbl_df['Last_Login_Date'])

# Get current date
current_date = pd.Timestamp.now()

# Create conditions for different time brackets
last_30_days = current_date - pd.Timedelta(days=30)
last_90_days = current_date - pd.Timedelta(days=90) 
last_year = current_date - pd.Timedelta(days=365)

# Create Login_Bracket column
def get_login_bracket(login_date, registration_date):
    if pd.isna(login_date):
        if pd.notna(registration_date):
            return 'More than a Year'
        return 'Not Registered'
    elif login_date >= last_30_days:
        return 'Last 30 Days'
    elif login_date >= last_90_days:
        return 'Last 90 Days'
    elif login_date >= last_year:
        return 'Previous Year'
    else:
        return 'More than a Year'

ntb_reg_tbl_df['Login_Bracket'] = ntb_reg_tbl_df.apply(
    lambda x: get_login_bracket(x['Last_Login_Date'], x['Registration_Date']), 
    axis=1
)

# Print value counts to see distribution
print("\nLogin Bracket Distribution:")
value_counts = ntb_reg_tbl_df['Login_Bracket'].value_counts(dropna=False)
print(value_counts)
print(f"\nTotal: {value_counts.sum()}")
# Create a bar chart comparing months between 2024 and 2025
# First extract year and month from Open_Date_
ntb_reg_tbl_df['Year'] = ntb_reg_tbl_df['Open_Date_'].dt.year
ntb_reg_tbl_df['Month'] = ntb_reg_tbl_df['Open_Date_'].dt.month

# Get monthly counts for each year
monthly_counts = ntb_reg_tbl_df.groupby(['Year', 'Month']).size().reset_index(name='Count')

# Filter for just 2024 and 2025
monthly_counts = monthly_counts[monthly_counts['Year'].isin([2024, 2025])]

# Create the bar plot
plt.figure(figsize=(12, 6))
bar_width = 0.35

# Plot bars for each year
plt.bar(monthly_counts[monthly_counts['Year']==2024]['Month'] - bar_width/2, 
        monthly_counts[monthly_counts['Year']==2024]['Count'],
        bar_width, label='2024')
plt.bar(monthly_counts[monthly_counts['Year']==2025]['Month'] + bar_width/2,
        monthly_counts[monthly_counts['Year']==2025]['Count'], 
        bar_width, label='2025')

plt.xlabel('Month')
plt.ylabel('Number of Accounts')
plt.title('Monthly Account Distribution: 2024 vs 2025')
plt.legend()
plt.xticks(range(1,13))
plt.grid(True, alpha=0.3)
# Save plot with datetime stamp
current_time = pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')
save_path = os.path.join('./Figures', f'monthly_accounts_{current_time}.png')

# Create Figures directory if it doesn't exist
os.makedirs('./Figures', exist_ok=True)

plt.savefig(save_path, bbox_inches='tight', dpi=300)

plt.show()

# Print summary statistics
print("\nMonthly Account Summary:")
print(monthly_counts.sort_values(['Year', 'Month']))


In [ ]:
# Convert date columns to datetime with consistent format handling
ntb_reg_tbl_df['Registration_Date'] = pd.to_datetime(ntb_reg_tbl_df['Registration_Date'], errors='coerce')
ntb_reg_tbl_df['Open_Date_'] = pd.to_datetime(ntb_reg_tbl_df['Open_Date_'], errors='coerce')

# Create mask for valid onboarding calculation (both dates exist and registration after opening)
valid_onboarding_mask = (~ntb_reg_tbl_df['Registration_Date'].isna()) & (~ntb_reg_tbl_df['Open_Date_'].isna()) & (ntb_reg_tbl_df['Registration_Date'] >= ntb_reg_tbl_df['Open_Date_'])

# Calculate days to onboard
ntb_reg_tbl_df.loc[valid_onboarding_mask, 'Days_to_Onboard'] = (
    ntb_reg_tbl_df.loc[valid_onboarding_mask, 'Registration_Date'] - 
    ntb_reg_tbl_df.loc[valid_onboarding_mask, 'Open_Date_']
).dt.days

# Get overall average
avg_days_to_onboard = ntb_reg_tbl_df['Days_to_Onboard'].mean()
print(f"Average days to onboard: {avg_days_to_onboard:.2f}")

# Get average by region
region_onboarding = ntb_reg_tbl_df.groupby('REGION_DESC')['Days_to_Onboard'].mean().reset_index()
print("\nAverage onboarding time by region:")
print(region_onboarding.sort_values('Days_to_Onboard'))

In [ ]:
# Create bins for customers with login data
login_mask = ~ntb_reg_tbl_df['Last_Login_Date'].isna()
current_date = pd.Timestamp.now()

# Calculate days since last login
ntb_reg_tbl_df.loc[login_mask, 'Days_Since_Last_Login'] = (
    current_date - ntb_reg_tbl_df.loc[login_mask, 'Last_Login_Date']
).dt.days

# Create frequency categories 
def get_login_frequency(days):
    if pd.isna(days):
        return 'No Login'
    elif days <= 7:
        return 'Weekly'
    elif days <= 30:
        return 'Monthly'
    elif days <= 90:
        return 'Quarterly'
    else:
        return 'Inactive'

ntb_reg_tbl_df['Login_Frequency'] = ntb_reg_tbl_df['Days_Since_Last_Login'].apply(get_login_frequency)

# Update 'No Login' to 'Not Registered' when Registration_Remarks is 'Not Registered'
not_registered_mask = (ntb_reg_tbl_df['Login_Frequency'] == 'No Login') & (ntb_reg_tbl_df['Registration_Remarks'] == 'Not Registered')
ntb_reg_tbl_df.loc[not_registered_mask, 'Login_Frequency'] = 'Not Registered'

# Verify the changes
print("\nUpdated Login Frequency Distribution:")
updated_freq_dist = ntb_reg_tbl_df['Login_Frequency'].value_counts().reset_index()
updated_freq_dist.columns = ['Frequency', 'Count']
print(updated_freq_dist)

# Calculate frequency distribution
login_frequency_dist = ntb_reg_tbl_df['Login_Frequency'].value_counts().reset_index()
login_frequency_dist.columns = ['Frequency', 'Count']

print("\nLogin Frequency Distribution:")
print(login_frequency_dist)

# Calculate by region
region_frequency = pd.crosstab(
    ntb_reg_tbl_df['REGION_DESC'], 
    ntb_reg_tbl_df['Login_Frequency'],
    normalize='index'
) * 100

print("\nLogin Frequency by Region (%):")
print(region_frequency)


# Define a clean order for the categories
category_order = ['Weekly', 'Monthly', 'Quarterly', 'Inactive', 'No Login', 'Not Registered']
# Filter to only include categories in our data
valid_categories = [cat for cat in category_order if cat in login_frequency_dist['Frequency'].values]
# Filter and sort the dataframe
sorted_dist = login_frequency_dist[login_frequency_dist['Frequency'].isin(valid_categories)]
sorted_dist = sorted_dist.set_index('Frequency').reindex(valid_categories).reset_index()


# Calculate login frequency by RGM
rgm_frequency = pd.crosstab(
    ntb_reg_tbl_df['RGM'],
    ntb_reg_tbl_df['Login_Frequency'],
    normalize='index'
) * 100

# Calculate active user rate (Weekly + Monthly) by RGM
if 'Weekly' in rgm_frequency.columns and 'Monthly' in rgm_frequency.columns:
    active_cols = ['Weekly', 'Monthly']
    rgm_active_rate = rgm_frequency[active_cols].sum(axis=1).sort_values(ascending=False)
    
    print("\nTop 10 RGMs by Active User Rate (Weekly + Monthly):")
    print(rgm_active_rate.head(10).round(1))
    
    # Save RGM active user rates
    rgm_active_rate = rgm_active_rate.reset_index()
    rgm_active_rate.columns = ['RGM', 'Active_User_Rate']
    rgm_active_rate.to_excel('./Notebook_reports/rgm_active_user_rates.xlsx', index=False)


In [ ]:
# Make sure dates are in datetime format
ntb_reg_tbl_df['Registration_Date'] = pd.to_datetime(ntb_reg_tbl_df['Registration_Date'], errors='coerce')
ntb_reg_tbl_df['Open_Date_'] = pd.to_datetime(ntb_reg_tbl_df['Open_Date_'], errors='coerce')

# Initialize Days_to_Onboard column as NaN
ntb_reg_tbl_df['Days_to_Onboard'] = np.nan

# Create a mask for eligible records:
# 1. Not 'Already Registered'
# 2. Has both dates
# 3. Registration date is after or equal to open date
eligible_mask = (
    (ntb_reg_tbl_df['Registration_Remarks'] != 'Already Registered') & 
    (~ntb_reg_tbl_df['Registration_Date'].isna()) & 
    (~ntb_reg_tbl_df['Open_Date_'].isna()) & 
    (ntb_reg_tbl_df['Registration_Date'] >= ntb_reg_tbl_df['Open_Date_'])
)

# Calculate days to onboard only for eligible records
ntb_reg_tbl_df.loc[eligible_mask, 'Days_to_Onboard'] = (
    ntb_reg_tbl_df.loc[eligible_mask, 'Registration_Date'] - 
    ntb_reg_tbl_df.loc[eligible_mask, 'Open_Date_']
).dt.days

# Create days_to_onboard categories
def get_onboarding_bracket(days):
    if pd.isna(days):
        return 'Not Registered'
    elif days <= 5:
        return '5 days or less'
    elif days <= 10:
        return '6-10 days'
    elif days <= 30:
        return '11-30 days'
    elif days <= 180:
        return '1-6 months'
    else:
        return 'More than 6 months'

# Apply the binning function to eligible records
ntb_reg_tbl_df['Onboarding_Time_Bracket'] = 'Not Registered'  # Default value
ntb_reg_tbl_df.loc[eligible_mask, 'Onboarding_Time_Bracket'] = ntb_reg_tbl_df.loc[eligible_mask, 'Days_to_Onboard'].apply(get_onboarding_bracket)

# Update Onboarding_Time_Bracket to "Already Registered" for customers with Registration_Remarks = "Already Registered"
already_registered_mask = ntb_reg_tbl_df['Registration_Remarks'] == 'Already Registered'
ntb_reg_tbl_df.loc[already_registered_mask, 'Onboarding_Time_Bracket'] = 'Already Registered'

# Summary statistics
valid_days = ntb_reg_tbl_df.loc[eligible_mask, 'Days_to_Onboard']
print(f"\nDays to Onboard - Summary Statistics:")
print(f"Count: {valid_days.count()}")
print(f"Mean: {valid_days.mean():.2f} days")
print(f"Median: {valid_days.median():.0f} days")
print(f"Min: {valid_days.min():.0f} days")
print(f"Max: {valid_days.max():.0f} days")

# Distribution of onboarding time brackets
print("\nOnboarding Time Bracket Distribution:")
onboard_dist = ntb_reg_tbl_df['Onboarding_Time_Bracket'].value_counts().sort_index()
print(onboard_dist)

# Exclude 'Not Registered' and 'Already Registered' for percentage calculation of normal onboarding times
normal_onboarding_mask = eligible_mask
onboard_dist_pct = ntb_reg_tbl_df.loc[normal_onboarding_mask, 'Onboarding_Time_Bracket'].value_counts(normalize=True) * 100
print("\nOnboarding Time Bracket Distribution (% of normally registered customers):")
print(onboard_dist_pct.sort_index().round(2))

# Create a filtered dataframe for non-'Already Registered' users for regional and RGM analysis
# This is needed to get accurate regional and RGM insights about normal onboarding patterns
real_ntb_reg_tbl_df = ntb_reg_tbl_df[ntb_reg_tbl_df['Registration_Remarks'] != 'Already Registered']

# Regional analysis of onboarding time brackets
region_onboard = pd.crosstab(
    real_ntb_reg_tbl_df['REGION_DESC'],
    real_ntb_reg_tbl_df['Onboarding_Time_Bracket'],
    normalize='index'
) * 100

print("\nDays to Onboard by Region (%):")
if 'Not Registered' in region_onboard.columns:
    # Move 'Not Registered' to the end for better readability
    cols = [col for col in region_onboard.columns if col != 'Not Registered'] + ['Not Registered']
    region_onboard = region_onboard[cols]

print(region_onboard.round(1))

# Save the region analysis
region_onboard.to_excel('./Notebook_reports/region_onboarding_time.xlsx')

# Additional analysis: RGM-level onboarding performance
rgm_onboard = pd.crosstab(
    real_ntb_reg_tbl_df['RGM'],
    real_ntb_reg_tbl_df['Onboarding_Time_Bracket'],
    normalize='index'
) * 100

# Calculate key metrics for RGMs
rgm_metrics = real_ntb_reg_tbl_df.groupby('RGM').agg(
    Total_Accounts=('CUSTOMER_NO', 'count'),
    Registered_Count=('Registration_Date', lambda x: x.notna().sum()),
    Average_Days_to_Onboard=('Days_to_Onboard', 'mean')
).reset_index()

# Calculate registration rate
rgm_metrics['Registration_Rate'] = (rgm_metrics['Registered_Count'] / rgm_metrics['Total_Accounts'] * 100).round(1)

# Calculate % of quick onboarders (<=10 days)
quick_onboarders = pd.crosstab(
    real_ntb_reg_tbl_df['RGM'],
    real_ntb_reg_tbl_df['Onboarding_Time_Bracket'].isin(['5 days or less', '6-10 days']),
    normalize='index'
) * 100

if True in quick_onboarders.columns:
    rgm_metrics['Quick_Onboarding_Rate'] = quick_onboarders[True].round(1)
else:
    rgm_metrics['Quick_Onboarding_Rate'] = 0

# Sort by Quick Onboarding Rate
rgm_metrics = rgm_metrics.sort_values('Quick_Onboarding_Rate', ascending=False)

print("\nRGM Performance in Onboarding:")
print(rgm_metrics.head(10))  # Top 10 RGMs

# Save RGM analysis
rgm_metrics.to_excel('./Notebook_reports/rgm_onboarding_performance.xlsx', index=False)


In [57]:
# Extract month from registration date
ntb_reg_tbl_df['Registration_Month'] = ntb_reg_tbl_df['Registration_Date'].dt.to_period('M')

# Calculate monthly onboarding times
monthly_onboarding = ntb_reg_tbl_df.groupby('Registration_Month')['Days_to_Onboard'].agg(['mean', 'median', 'count']).reset_index()
monthly_onboarding = monthly_onboarding.sort_values('Registration_Month')



In [58]:
# Branch performance metrics
branch_metrics = ntb_reg_tbl_df.groupby('BRANCH_NAME').agg(
    Total_Accounts=('CUSTOMER_NO', 'count'),
    Registered_Count=('Registration_Date', lambda x: x.notna().sum()),
    Avg_Days_to_Onboard=('Days_to_Onboard', 'mean')
).reset_index()

# Calculate registration rate and add REGION_DESC
branch_metrics['Registration_Rate'] = (branch_metrics['Registered_Count'] / branch_metrics['Total_Accounts'] * 100).round(1)

# Add region info to branch metrics (based on a single branch's region)
branch_region = ntb_reg_tbl_df.groupby('BRANCH_NAME')['REGION_DESC'].first().reset_index()
branch_metrics = pd.merge(branch_metrics, branch_region, on='BRANCH_NAME')

# Identify top and bottom performing branches
top_branches = branch_metrics.sort_values('Registration_Rate', ascending=False).head(20)
bottom_branches = branch_metrics.sort_values('Registration_Rate').head(20)

# Save branch metrics
branch_metrics.to_excel('./Notebook_reports/branch_onboarding_performance.xlsx', index=False)

In [ ]:
# Define funnel stages
funnel_data = {
    'Stage': ['Account Opening', 'Mobile Registration', 'Active in Last 30 Days', 'Weekly Active Users'],
    'Count': [
        len(ntb_reg_tbl_df),  # All accounts
        ntb_reg_tbl_df['Registration_Remarks'].isin(['Registered', 'Already Registered']).sum(),  # All registered
        (ntb_reg_tbl_df['Login_Bracket'] == 'Last 30 Days').sum(),  # Active in last 30 days
        (ntb_reg_tbl_df['Login_Frequency'] == 'Weekly').sum()  # Log in weekly
    ]
}

# Create dataframe
funnel_df = pd.DataFrame(funnel_data)

# Calculate percentage of total accounts
funnel_df['Percentage'] = (funnel_df['Count'] / funnel_df['Count'][0] * 100).round(1)

# Calculate stage-to-stage conversion rates
funnel_df['Conversion_Rate'] = 100.0  # Initialize with a default value for all rows

# Update conversion rates for rows after the first one
for i in range(1, len(funnel_df)):
    # Conversion from previous stage to current stage
    funnel_df.loc[i, 'Conversion_Rate'] = (funnel_df['Count'][i] / funnel_df['Count'][i-1] * 100).round(1)

# Add stage drop-off
funnel_df['Drop_Off'] = 100 - funnel_df['Conversion_Rate']

print("Customer Journey Funnel:")
print(funnel_df)


# Export to Excel for dashboard
funnel_df.to_excel('./Notebook_reports/customer_journey_funnel.xlsx', index=False)

In [ ]:
# Create funnel by region
regions = ntb_reg_tbl_df['REGION_DESC'].unique()
region_funnels = []

for region in regions:
    region_data = ntb_reg_tbl_df[ntb_reg_tbl_df['REGION_DESC'] == region]
    
    funnel = {
        'Region': region,
        'Total_Accounts': len(region_data),
        'Registered': region_data['Registration_Remarks'].isin(['Registered', 'Already Registered']).sum(),
        'Active_30_Days': (region_data['Login_Bracket'] == 'Last 30 Days').sum(),
        'Weekly_Users': (region_data['Login_Frequency'] == 'Weekly').sum()
    }
    
    region_funnels.append(funnel)

region_funnel_df = pd.DataFrame(region_funnels)

# Calculate conversion rates
region_funnel_df['Registration_Rate'] = (region_funnel_df['Registered'] / region_funnel_df['Total_Accounts'] * 100).round(1)
region_funnel_df['Activation_Rate'] = (region_funnel_df['Active_30_Days'] / region_funnel_df['Registered'] * 100).round(1)
region_funnel_df['Weekly_Usage_Rate'] = (region_funnel_df['Weekly_Users'] / region_funnel_df['Active_30_Days'] * 100).round(1)

# Sort by Registration Rate
region_funnel_df = region_funnel_df.sort_values('Registration_Rate', ascending=False)

# Print the regional funnel analysis
print("\nRegional Funnel Analysis:")
print(region_funnel_df)

# Save to Excel
region_funnel_df.to_excel('./Notebook_reports/regional_funnel_analysis.xlsx', index=False)


In [ ]:
# Group by account opening month
ntb_reg_tbl_df['Open_Month'] = pd.to_datetime(ntb_reg_tbl_df['Open_Date_']).dt.to_period('M')

# Create monthly cohorts and track their progression through the funnel
monthly_cohorts = []

for month in sorted(ntb_reg_tbl_df['Open_Month'].unique()):
    cohort = ntb_reg_tbl_df[ntb_reg_tbl_df['Open_Month'] == month]
    
    # Calculate funnel metrics for this cohort
    metrics = {
        'Cohort_Month': str(month),  # Convert to string for display
        'Total_Accounts': len(cohort),
        'Registered_30d': cohort[
            (cohort['Registration_Date'].notna()) & 
            ((cohort['Registration_Date'] - cohort['Open_Date_']).dt.days <= 30)
        ].shape[0],
        'Registered_90d': cohort[
            (cohort['Registration_Date'].notna()) & 
            ((cohort['Registration_Date'] - cohort['Open_Date_']).dt.days <= 90)
        ].shape[0]
    }
    
    monthly_cohorts.append(metrics)

monthly_funnel_df = pd.DataFrame(monthly_cohorts)
monthly_funnel_df['30d_Registration_Rate'] = (monthly_funnel_df['Registered_30d'] / monthly_funnel_df['Total_Accounts'] * 100).round(1)
monthly_funnel_df['90d_Registration_Rate'] = (monthly_funnel_df['Registered_90d'] / monthly_funnel_df['Total_Accounts'] * 100).round(1)

# Print the monthly cohort analysis
print("\nMonthly Cohort Analysis:")
print(monthly_funnel_df)

# Save to Excel
monthly_funnel_df.to_excel('./Notebook_reports/monthly_cohort_analysis.xlsx', index=False)


In [ ]:
# Create summary metrics for dashboard
summary_metrics = {
    'Metric': [
        'Total Accounts',
        'Registered Accounts',
        'Registration Rate',
        'Active 30 Days',
        'Active Rate',
        'Avg Days to Onboard',
        'Already Registered',
        'Quick Onboarding (≤10 days)'
    ],
    'Value': [
        len(ntb_reg_tbl_df),
        ntb_reg_tbl_df['Registration_Remarks'].isin(['Registered', 'Already Registered']).sum(),
        ntb_reg_tbl_df['Registration_Remarks'].isin(['Registered', 'Already Registered']).sum() / len(ntb_reg_tbl_df) * 100,
        (ntb_reg_tbl_df['Login_Bracket'] == 'Last 30 Days').sum(),
        (ntb_reg_tbl_df['Login_Bracket'] == 'Last 30 Days').sum() / ntb_reg_tbl_df['Registration_Remarks'].isin(['Registered', 'Already Registered']).sum() * 100,
        ntb_reg_tbl_df['Days_to_Onboard'].mean(),
        (ntb_reg_tbl_df['Registration_Remarks'] == 'Already Registered').sum(),
        ntb_reg_tbl_df['Onboarding_Time_Bracket'].isin(['5 days or less', '6-10 days']).sum() / ntb_reg_tbl_df['Registration_Remarks'].isin(['Registered', 'Already Registered']).sum() * 100
    ]
}

# Convert to DataFrame for easy export
summary_df = pd.DataFrame(summary_metrics)
summary_df['Value'] = summary_df['Value'].round(2)
print("\nSummary Metrics:")
print(summary_df)

# Export summary metrics to Excel
summary_df.to_excel('./Notebook_reports/Mobile_Banking_Summary_Metrics.xlsx', index=False)

In [63]:
# # Save the updated main dataframe with onboarding metrics
ntb_reg_tbl_df.to_excel('./Notebook_reports/NTB_Reg_Summary_with_Onboarding.xlsx', index=False)